In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER,LATITUDE_FORMATTER
import os,errno
import sys
import cartopy.feature as cfeature
import matplotlib.ticker as mticker
import pandas as pd
import datetime as dt
from scipy.ndimage.measurements import label
from math import sin, cos, sqrt, atan2, radians
import geopy.distance
%matplotlib inline

dir2='/thorncroftlab_rit/ahenny/rain/'
dir1='/thorncroftlab_rit/ahenny/rain/US/ghcnd_all/'
dir='/thorncroftlab_rit/ahenny/rain/DISSERTATION_SCRIPTS_RESULTS/'
#plots trends in synoptic quantities on EP days, independent of EP day frequency

In [ ]:
season='fall'
thresh_choose=99
area_threshold_choose=80
season_choose=season

ds=xr.open_dataset(dir+'extreme_days_ghcnd_'+str(thresh_choose)+'_'+str(area_threshold_choose)+'_'+season_choose+'.nc')

lats=ds['lats'].values.tolist()
lons=ds['lons'].values.tolist()
lons=[x+360. for x in lons]
dates=ds['dates'].values
dates_unique=list(set(dates))
dates_unique=pd.DatetimeIndex(dates_unique).sort_values()
stations=ds['stations'].values.tolist()
obs=ds['obs'].values.tolist()
years=[x.year for x in dates_unique]
print(dates_unique)
yrs_neusa=np.arange(1979,2020,1)

lon_range=np.arange(-115,-30,0.25)
lat_range=np.arange(15,60,0.25)

ds1=xr.open_dataset(dir+'era_5_annual_ivtx_neusa.nc')
print(ds1)
ivtx=ds1['p71.162'].sel(latitude=lat_range,longitude=lon_range)

ds2=xr.open_dataset(dir+'era_5_annual_ivty_neusa.nc')
ivty=ds2['p72.162'].sel(latitude=lat_range,longitude=lon_range)

ds3=xr.open_dataset(dir+'era_5_annual_z1000_neusa.nc')
z_1000=ds3.z.sel(latitude=lat_range,longitude=lon_range)

ds6=xr.open_dataset(dir+'era_5_annual_z500_neusa.nc')
z_500=ds6.z.sel(latitude=lat_range,longitude=lon_range)

ds5=xr.open_dataset(dir+'era_5_annual_mslp_neusa.nc')
slp=ds5['msl'].sel(latitude=lat_range,longitude=lon_range)

ds4=xr.open_dataset(dir+'neusa_ep_days_stats_seasonal_ep_days_'+season+'.nc')

ar_yesno4=ds4['ar_yesno'].values.tolist()
ivt_yesno4=ds4['ivt_yesno'].values.tolist()
tc_yesno4=ds4['tc_yesno'].values.tolist()
other_yesno4=ds4['other_yesno'].values.tolist()
tc_linked_ar_yesno4=ds4['tc_linked_ar_yesno'].values.tolist()
tc_linked_ivt_yesno4=ds4['tc_linked_ivt_yesno'].values.tolist()
tc_remnant_linked_ivt_yesno4=ds4['tc_remnant_linked_ivt_yesno'].values.tolist()
tc_remnant_ar_combo_yesno4=ds4['tc_remnant_ar_combo_yesno'].values.tolist()
tc_remnant_linked_ar_yesno4=ds4['tc_remnant_linked_ar_yesno'].values.tolist()
tc_remnants_yesno4=ds4['tc_remnants_yesno'].values.tolist()
tc_ar_combo_yesno4=ds4['tc_ar_combo_yesno'].values.tolist()

print(sum(ar_yesno4))

In [ ]:
if 1==1:
    dsa=xr.open_dataset(dir+'neusa_ar_climo_fall.nc')
    ar=dsa.ar
    tc_trop=dsa.tc_trop
    tc_nontrop=dsa.tc_nontrop
    labeled=dsa.labeled
print(season)
if 1==0:
    dsa=xr.open_dataset(dir+'neusa_ar_points_'+season+'_var95const.nc')
    dsa['longitude']=dsa.longitude
    ar=dsa.ar
    tc_trop=dsa.tc_trop
    tc_nontrop=dsa.tc_nontrop
    labeled=dsa.labeled

In [ ]:
zipped_t1=list(zip(dates_unique,ar_yesno4))#ar-related = green
zipped_t2=list(zip(dates_unique,tc_linked_ar_yesno4))
zipped_t3=list(zip(dates_unique,tc_remnant_linked_ar_yesno4))
zipped_t4=list(zip(dates_unique,tc_yesno4))#tc-related = blue
zipped_t5=list(zip(dates_unique,tc_ar_combo_yesno4))
zipped_t6=list(zip(dates_unique,tc_remnant_ar_combo_yesno4))
zipped_t7=list(zip(dates_unique,tc_remnants_yesno4))
zipped_t8=list(zip(dates_unique,ivt_yesno4))#other IVT-related = grey
zipped_t9=list(zip(dates_unique,tc_linked_ivt_yesno4))
zipped_t10=list(zip(dates_unique,tc_remnant_linked_ivt_yesno4))
zipped_t11=list(zip(dates_unique,other_yesno4))#unspecified = brown

dates_t1=[x[0] for x in zipped_t1 if x[1]==1]
dates_t2=[x[0] for x in zipped_t2 if x[1]==1]
dates_t3=[x[0] for x in zipped_t3 if x[1]==1]
dates_t4=[x[0] for x in zipped_t4 if x[1]==1]
dates_t5=[x[0] for x in zipped_t5 if x[1]==1]
dates_t6=[x[0] for x in zipped_t6 if x[1]==1]
dates_t7=[x[0] for x in zipped_t7 if x[1]==1]
dates_t8=[x[0] for x in zipped_t8 if x[1]==1]
dates_t9=[x[0] for x in zipped_t9 if x[1]==1]
dates_t10=[x[0] for x in zipped_t10 if x[1]==1]
dates_t11=[x[0] for x in zipped_t11 if x[1]==1]


dates_t1=[pd.to_datetime(x)+dt.timedelta(hours=0) for x in dates_t1]
dates_t2=[pd.to_datetime(x)+dt.timedelta(hours=0) for x in dates_t2]
dates_t3=[pd.to_datetime(x)+dt.timedelta(hours=0) for x in dates_t3]
dates_t4=[pd.to_datetime(x)+dt.timedelta(hours=0) for x in dates_t4]
dates_t5=[pd.to_datetime(x)+dt.timedelta(hours=0) for x in dates_t5]
dates_t6=[pd.to_datetime(x)+dt.timedelta(hours=0) for x in dates_t6]
dates_t7=[pd.to_datetime(x)+dt.timedelta(hours=0) for x in dates_t7]
dates_t8=[pd.to_datetime(x)+dt.timedelta(hours=0) for x in dates_t8]
dates_t9=[pd.to_datetime(x)+dt.timedelta(hours=0) for x in dates_t9]
dates_t10=[pd.to_datetime(x)+dt.timedelta(hours=0) for x in dates_t10]
dates_t11=[pd.to_datetime(x)+dt.timedelta(hours=0) for x in dates_t11]
print(dates_t1)

years_t1=[x.year for x in dates_t1]
print(len(years_t1))
print(years_t1)
years_t2=[x.year for x in dates_t2]
years_t3=[x.year for x in dates_t3]
years_t4=[x.year for x in dates_t4]
years_t5=[x.year for x in dates_t5]
years_t6=[x.year for x in dates_t6]
years_t7=[x.year for x in dates_t7]
years_t8=[x.year for x in dates_t8]
years_t9=[x.year for x in dates_t9]
years_t10=[x.year for x in dates_t10]
years_t11=[x.year for x in dates_t11]

dates_ar=dates_t1+dates_t2+dates_t3
dates_tc=dates_t4+dates_t5+dates_t6+dates_t7
dates_other=dates_t8+dates_t9+dates_t10+dates_t11

years_ar=years_t1+years_t2+years_t3
years_tc=years_t4+years_t5+years_t6+years_t7
years_other=years_t8+years_t9+years_t10+years_t11

years_ar=np.sort(years_ar).tolist()
years_tc=np.sort(years_tc).tolist()
years_other=np.sort(years_other).tolist()

years_all=[x.year for x in dates_unique]
print(len(dates_ar))

In [ ]:
dates_choose=dates_tc
for i in range(len(dates_choose)):
    print(i)
    date=dates_choose[i]
    date_range=[date+dt.timedelta(hours=6*x) for x in range(4)]
    date_range=pd.DatetimeIndex(date_range).values#date range for IVT selection
    
    #date_range=date
    ivtx_sel=ivtx.sel(time=date_range)
    ivty_sel=ivty.sel(time=date_range)
    ivt_mag_sel=np.sqrt(ivtx_sel**2+ivty_sel**2)
    slp_sel=slp.sel(time=date_range)
    ar_sel=ar.sel(time=date_range)
    ar_sel=ar_sel.fillna(0)
    z_1000_sel=z_1000.sel(time=date_range)
    z_500_sel=z_500.sel(time=date_range)
    thickness_sel=z_500_sel-z_1000_sel
    print(thickness_sel)

    if 1==1:
        ivt_mag_mean=ivt_mag_sel.mean(dim='time',skipna=True)
        ivtx_mean=ivtx_sel.mean(dim='time',skipna=True)
        ivty_mean=ivty_sel.mean(dim='time',skipna=True)
        slp_mean=slp_sel.mean(dim='time',skipna=True)
        thickness_mean=thickness_sel.mean(dim='time',skipna=True)
        ar_mean=ar_sel.mean(dim='time',skipna=True)
    if 1==0:    
        ivt_mag_mean=ivt_mag_sel
        ivtx_mean=ivtx_sel
        ivty_mean=ivty_sel
        slp_mean=slp_sel
        thickness_mean=thickness_sel
        ar_mean=ar_sel
    
    if i==0:
        ivt_days=ivt_mag_mean
        ivtx_days=ivtx_mean
        ivty_days=ivty_mean
        slp_days=slp_mean
        thickness_days=thickness_mean
        ar_days=ar_mean
    else:
        ivt_days=xr.concat([ivt_days,ivt_mag_mean],dim='time')
        ivtx_days=xr.concat([ivtx_days,ivtx_mean],dim='time')
        ivty_days=xr.concat([ivty_days,ivty_mean],dim='time')
        slp_days=xr.concat([slp_days,slp_mean],dim='time')
        thickness_days=xr.concat([thickness_days,thickness_mean],dim='time')
        ar_days=xr.concat([ar_days,ar_mean],dim='time')
print(thickness_days)

In [ ]:
years_select=years_tc
import scipy.stats as st
if 1==1:
    slopes_array_ivt=xr.zeros_like(ivtx_days[0,:,:])
    sigs_array_ivt=xr.zeros_like(ivtx_days[0,:,:])
    slopes_array_ivtx=xr.zeros_like(ivtx_days[0,:,:])
    sigs_array_ivtx=xr.zeros_like(ivtx_days[0,:,:])
    slopes_array_ivty=xr.zeros_like(ivty_days[0,:,:])
    sigs_array_ivty=xr.zeros_like(ivty_days[0,:,:])
    slopes_array_slp=xr.zeros_like(slp_days[0,:,:])
    sigs_array_slp=xr.zeros_like(slp_days[0,:,:])
    slopes_array_thickness=xr.zeros_like(thickness_days[0,:,:])
    sigs_array_thickness=xr.zeros_like(thickness_days[0,:,:])
    slopes_array_ar=xr.zeros_like(ar_days[0,:,:])
    sigs_array_ar=xr.zeros_like(ar_days[0,:,:])
    
    for i in range(ar.latitude.size):
        print(i)
        for j in range(ar.longitude.size):
            time_series=ar_days[:,i,j].values.tolist()
            linreg=st.linregress(years_select,time_series)
            slope=linreg[0]
            p_value=linreg[3]
            slopes_array_ar[i,j]=slope
            sigs_array_ar[i,j]=p_value
            
    for i in range(ivtx_days.latitude.size):
        print(i)
        for j in range(ivtx_days.longitude.size):
            time_series=ivt_days[:,i,j].values.tolist()
            linreg=st.linregress(years_select,time_series)
            slope=linreg[0]
            p_value=linreg[3]
            slopes_array_ivt[i,j]=slope
            sigs_array_ivt[i,j]=p_value
            
            time_series=ivtx_days[:,i,j].values.tolist()
            linreg=st.linregress(years_select,time_series)
            slope=linreg[0]
            p_value=linreg[3]
            slopes_array_ivtx[i,j]=slope
            sigs_array_ivtx[i,j]=p_value
            
            time_series=ivty_days[:,i,j].values.tolist()
            linreg=st.linregress(years_select,time_series)
            slope=linreg[0]
            p_value=linreg[3]
            slopes_array_ivty[i,j]=slope
            sigs_array_ivty[i,j]=p_value
            
    for i in range(slp_days.latitude.size):
        print(i)
        for j in range(slp_days.longitude.size):
            time_series=slp_days[:,i,j].values.tolist()
            linreg=st.linregress(years_select,time_series)
            slope=linreg[0]
            p_value=linreg[3]
            slopes_array_slp[i,j]=slope
            sigs_array_slp[i,j]=p_value
            
    for i in range(thickness_days.latitude.size):
        print(i)
        for j in range(thickness_days.longitude.size):
            time_series=thickness_days[:,i,j].values.tolist()
            linreg=st.linregress(years_select,time_series)
            slope=linreg[0]
            p_value=linreg[3]
            slopes_array_thickness[i,j]=slope
            sigs_array_thickness[i,j]=p_value            

In [ ]:
#to draw grid lines on Lambert Conformal projection; 
#CREDIT ajdawson on GitHub https://gist.github.com/ajdawson/dd536f786741e987ae4e

from copy import copy
import shapely.geometry as sgeom
def find_side(ls, side):
    """
    Given a shapely LineString which is assumed to be rectangular, return the
    line corresponding to a given side of the rectangle.
    
    """
    minx, miny, maxx, maxy = ls.bounds
    points = {'left': [(minx, miny), (minx, maxy)],
              'right': [(maxx, miny), (maxx, maxy)],
              'bottom': [(minx, miny), (maxx, miny)],
              'top': [(minx, maxy), (maxx, maxy)],}
    return sgeom.LineString(points[side])


def lambert_xticks(ax, ticks):
    """Draw ticks on the bottom x-axis of a Lambert Conformal projection."""
    te = lambda xy: xy[0]
    lc = lambda t, n, b: np.vstack((np.zeros(n) + t, np.linspace(b[2], b[3], n))).T
    xticks, xticklabels = _lambert_ticks(ax, ticks, 'bottom', lc, te)
    ax.xaxis.tick_bottom()
    ax.set_xticks(xticks)
    ax.set_xticklabels([ax.xaxis.get_major_formatter()(xtick) for xtick in xticklabels])
    

def lambert_yticks(ax, ticks):
    """Draw ricks on the left y-axis of a Lamber Conformal projection."""
    te = lambda xy: xy[1]
    lc = lambda t, n, b: np.vstack((np.linspace(b[0], b[1], n), np.zeros(n) + t)).T
    yticks, yticklabels = _lambert_ticks(ax, ticks, 'left', lc, te)
    ax.yaxis.tick_left()
    ax.set_yticks(yticks)
    ax.set_yticklabels([ax.yaxis.get_major_formatter()(ytick) for ytick in yticklabels])
def _lambert_ticks(ax, ticks, tick_location, line_constructor, tick_extractor):
    """Get the tick locations and labels for an axis of a Lambert Conformal projection."""
    outline_patch = sgeom.LineString(ax.outline_patch.get_path().vertices.tolist())
    axis = find_side(outline_patch, tick_location)
    n_steps = 30
    extent = ax.get_extent(ccrs.PlateCarree())
    _ticks = []
    for t in ticks:
        xy = line_constructor(t, n_steps, extent)
        proj_xyz = ax.projection.transform_points(ccrs.Geodetic(), xy[:, 0], xy[:, 1])
        xyt = proj_xyz[..., :2]
        ls = sgeom.LineString(xyt.tolist())
        locs = axis.intersection(ls)
        if not locs:
            tick = [None]
        else:
            tick = tick_extractor(locs.xy)
        _ticks.append(tick[0])
    # Remove ticks that aren't visible:    
    ticklabels = copy(ticks)
    while True:
        try:
            index = _ticks.index(None)
        except ValueError:
            break
        _ticks.pop(index)
        ticklabels.pop(index)
    return _ticks, ticklabels

In [ ]:
#@author: Michael Schramm on GitHub
#This function is derived from code originally posted by Sat Kumar Tomer
#(satkumartomer@gmail.com)
#See also: http://vsp.pnnl.gov/help/Vsample/Design_Trend_Mann_Kendall.htm
from scipy.stats import norm
import scipy.stats as st
def mk_test(x, alpha=0.05):
    n = len(x)

    # calculate S
    s = 0
    for k in range(n-1):
        for j in range(k+1, n):
            s += np.sign(x[j] - x[k])

    # calculate the unique data
    unique_x, tp = np.unique(x, return_counts=True)
    g = len(unique_x)

    # calculate the var(s)
    if n == g:  # there is no tie
        var_s = (n*(n-1)*(2*n+5))/18
    else:  # there are some ties in data
        var_s = (n*(n-1)*(2*n+5) - np.sum(tp*(tp-1)*(2*tp+5)))/18

    if s > 0:
        z = (s - 1)/np.sqrt(var_s)
    elif s < 0:
        z = (s + 1)/np.sqrt(var_s)
    else: # s == 0:
        z = 0

    # calculate the p_value
    p = 2*(1-norm.cdf(abs(z)))  # two tail test
    h = abs(z) > norm.ppf(1-alpha/2)

    if (z < 0) and h:
        trend = 'decreasing'
    elif (z > 0) and h:
        trend = 'increasing'
    else:
        trend = 'no trend'

    return trend, h, p, z

In [ ]:
proj_map = ccrs.PlateCarree()
proj_map = ccrs.PlateCarree()
clon=-70
clat=35
proj_map = ccrs.LambertConformal(central_longitude=clon, central_latitude=clat)
fig = plt.figure(figsize=(24,16))
ax=plt.subplot(1,1,1,projection=proj_map)
# *must* call draw in order to get the axis boundary used to add ticks:

ax.coastlines(resolution='10m')
ax.add_feature(cfeature.STATES.with_scale('10m'),alpha=0.3)
ax.add_feature(cfeature.LAKES.with_scale('50m'))
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
    
ax.set_extent([-100,-45,19,55],crs=ccrs.PlateCarree())
ax.set_title('1000-500 hPa thickness',fontsize=52,pad=10)

Y=5

for i in range(int(slopes_array_thickness.longitude.size/Y)):
    for j in range(int(slopes_array_thickness.latitude.size/Y)):
        if sigs_array_thickness[Y*j,Y*i]<=0.05:
            if slopes_array_thickness[Y*j,Y*i].values>0:
                ax.plot(slopes_array_thickness.longitude.values[Y*i],slopes_array_thickness.latitude.values[Y*j],transform=ccrs.PlateCarree(),marker='+',color='r',mew=3.5,markersize=15,markerfacecolor="None")
            else:
                ax.plot(slopes_array_thickness.longitude.values[Y*i],slopes_array_thickness.latitude.values[Y*j],transform=ccrs.PlateCarree(),marker='_',color='b',mew=3.5,markersize=15,markerfacecolor="None")
        elif sigs_array_thickness[Y*j,Y*i]<=0.10:
            if slopes_array_thickness[Y*j,Y*i].values>0:
                ax.plot(slopes_array_thickness.longitude.values[Y*i],slopes_array_thickness.latitude.values[Y*j],transform=ccrs.PlateCarree(),marker='+',color='r',mew=1.75,markersize=15,markerfacecolor="None")
            else:
                ax.plot(slopes_array_thickness.longitude.values[Y*i],slopes_array_thickness.latitude.values[Y*j],transform=ccrs.PlateCarree(),marker='_',color='b',mew=1.75,markersize=15,markerfacecolor="None")

slp_means_all=slp_days.mean(dim='time',skipna=True)
print(thickness_days)
thickness_mean=thickness_days.mean(dim='time',skipna=True)

c1=ax.contour(thickness_mean.longitude,thickness_mean.latitude,thickness_mean/100.,linewidths=3.5,linestyles='--',transform=ccrs.PlateCarree(),colors=['b','b','b','b','b','r','r','r','r','r','r'],levels=np.arange(516,580,6))
#c=ax.contour(slp_means_all.longitude,slp_means_all.latitude,slp_means_all/100.,linewidths=1.5,transform=ccrs.PlateCarree(),colors='k',levels=np.arange(960,1040,4))

props = dict(boxstyle='round', facecolor='wheat', alpha=1.0)
ax.text(0.009, 0.9865,'n='+str(len(dates_choose)), transform=ax.transAxes, fontsize=34,verticalalignment='top', bbox=props,zorder=40)
#cbar.ax.tick_params(labelsize=32)
#cbar.set_label('kg m$^{-1}$ s$^{-1}$',fontsize=34,rotation=90,labelpad=15)
#plt.clabel(c,fmt='%.00f',fontsize=30)
plt.clabel(c1,fmt='%.00f',fontsize=30)

fig.canvas.draw()

# Define gridline locations and draw the lines using cartopy's built-in gridliner:
xticks = [-135,-130,-125,-120,-115,-110,-105,-95,-85,-75,-65,-55,-45,-40,-35,-30,-25,-20]
yticks = [20, 25, 30, 35, 40, 45]
ax.gridlines(xlocs=xticks, ylocs=yticks,alpha=0.5)
ax.tick_params(labelsize=34)
# Label the end-points of the gridlines using the custom tick makers:
ax.xaxis.set_major_formatter(LONGITUDE_FORMATTER) 
ax.yaxis.set_major_formatter(LATITUDE_FORMATTER)
lambert_xticks(ax, xticks)
lambert_yticks(ax, yticks)
#ax.text(lonmax+15,clat,'mm/day',ha='right',va='center',rotation=270,fontsize=14)
plt.show()

#done: fall, spring, summer, winter

In [ ]:
print(season)
fig.savefig(dir+'neusa_climo_event_trends_'+season+'11.png')

In [ ]:
proj_map = ccrs.PlateCarree()
proj_map = ccrs.PlateCarree()
clon=-70
clat=35
proj_map = ccrs.LambertConformal(central_longitude=clon, central_latitude=clat)
fig = plt.figure(figsize=(24,16))
ax=plt.subplot(1,1,1,projection=proj_map)
# *must* call draw in order to get the axis boundary used to add ticks:

#cax=ax.contourf(slopes_array_slp.longitude,slopes_array_slp.latitude,slopes_array_slp/100.,transform=ccrs.PlateCarree(),levels=np.arange(-0.3,0.33,0.03),extend='both',cmap=plt.cm.bwr)
#cbar=plt.colorbar(cax,pad=0,fraction=0.042)
#cbar.set_label('hPa yr$^{-1}$',fontsize=22,rotation=90,labelpad=0)

ax.coastlines(resolution='10m')
ax.add_feature(cfeature.STATES.with_scale('10m'),alpha=0.3)
ax.add_feature(cfeature.LAKES.with_scale('50m'))
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
    
ax.set_extent([-100,-45,19,55],crs=ccrs.PlateCarree())
#ax.set_title('ERA5 SLP and IVT trends',loc='left',fontsize=20)
ax.set_title('Mean sea level pressure',fontsize=52,pad=10)

Y=5
if 1==0:
    for i in range(int(slopes_array_slp.longitude.size/Y)):
        for j in range(int(slopes_array_slp.latitude.size/Y)):
            if sigs_array_slp[Y*j,Y*i]<=0.05:
                ax.plot(slopes_array_slp.longitude.values[Y*i],slopes_array_slp.latitude.values[Y*j],transform=ccrs.PlateCarree(),marker='o',color='w',mew=1.75,markersize=6,markerfacecolor="None")
            elif sigs_array_slp[Y*j,Y*i]<=0.10:
                ax.plot(slopes_array_slp.longitude.values[Y*i],slopes_array_slp.latitude.values[Y*j],transform=ccrs.PlateCarree(),marker='o',color='w',mew=0.5,markersize=6,markerfacecolor="None")

for i in range(int(slopes_array_slp.longitude.size/Y)):
    for j in range(int(slopes_array_slp.latitude.size/Y)):
        if sigs_array_slp[Y*j,Y*i]<=0.05:
            if slopes_array_slp[Y*j,Y*i].values>0:
                ax.plot(slopes_array_slp.longitude.values[Y*i],slopes_array_slp.latitude.values[Y*j],transform=ccrs.PlateCarree(),marker='+',color='k',mew=3.5,markersize=15,markerfacecolor="None")
            else:
                ax.plot(slopes_array_slp.longitude.values[Y*i],slopes_array_slp.latitude.values[Y*j],transform=ccrs.PlateCarree(),marker='_',color='k',mew=3.5,markersize=15,markerfacecolor="None")
        elif sigs_array_slp[Y*j,Y*i]<=0.10:
            if slopes_array_slp[Y*j,Y*i].values>0:
                ax.plot(slopes_array_slp.longitude.values[Y*i],slopes_array_slp.latitude.values[Y*j],transform=ccrs.PlateCarree(),marker='+',color='k',mew=1.75,markersize=15,markerfacecolor="None")
            else:
                ax.plot(slopes_array_slp.longitude.values[Y*i],slopes_array_slp.latitude.values[Y*j],transform=ccrs.PlateCarree(),marker='_',color='k',mew=1.75,markersize=15,markerfacecolor="None")
#for i in range(int(slopes_array.longitude.size/Y)):
#    for j in range(int(slopes_array.latitude.size/Y)):
#        if sigs_array_v[Y*j,Y*i]==1 or sigs_array_u[Y*j,Y*i]==1:
#            print('YES')
#            
#            ax.quiver(u1.longitude[Y*i],u1.longitude[Y*j],slopes_array_u[Y*j,Y*i],slopes_array_v[Y*j,Y*i])
#        else:
#            if sigs_array_v_10[Y*j,Y*i]==1 or sigs_array_u_10[Y*j,Y*i]==1:
#                print('YES')
#                ax.quiver(u1.longitude[Y*i],u1.longitude[Y*j],slopes_array_u[Y*j,Y*i],slopes_array_v[Y*j,Y*i])

sigs_array_ivtx_convert5=sigs_array_ivtx.where(sigs_array_ivtx<=0.05)
sigs_array_ivtx_convert5=sigs_array_ivtx_convert5/sigs_array_ivtx_convert5
sigs_array_ivty_convert5=sigs_array_ivty.where(sigs_array_ivty<=0.05)
sigs_array_ivty_convert5=sigs_array_ivty_convert5/sigs_array_ivty_convert5

sigs_array_ivtx_convert10=sigs_array_ivtx.where(sigs_array_ivtx<=0.10)
sigs_array_ivtx_convert10=sigs_array_ivtx_convert10/sigs_array_ivtx_convert10
sigs_array_ivty_convert10=sigs_array_ivty.where(sigs_array_ivty<=0.10)
sigs_array_ivty_convert10=sigs_array_ivty_convert10/sigs_array_ivty_convert10


sigs_array_u=sigs_array_ivtx_convert5
sigs_array_v=sigs_array_ivty_convert5
sigs_array_u10=sigs_array_ivtx_convert10
sigs_array_v10=sigs_array_ivty_convert10

slopes_array_u=slopes_array_ivtx
slopes_array_v=slopes_array_ivty

sig_uv=sigs_array_u+sigs_array_v
sig_uv10=sigs_array_u10+sigs_array_v10

X=8
slopes_array_u10_select=slopes_array_u.where(sig_uv10>0)
slopes_array_v10_select=slopes_array_v.where(sig_uv10>0)
#q=ax.quiver(ivtx.longitude[0::X].values,ivtx.latitude[0::X].values,slopes_array_u10_select[0::X,0::X].values,slopes_array_v10_select[0::X,0::X].values,transform=ccrs.PlateCarree(),units='inches',scale=10,width=0.04,alpha=0.45,zorder=30)

slopes_array_u_select=slopes_array_u.where(sig_uv>0)
slopes_array_v_select=slopes_array_v.where(sig_uv>0)
#q=ax.quiver(ivtx.longitude[0::X].values,ivtx.latitude[0::X].values,slopes_array_u_select[0::X,0::X].values,slopes_array_v_select[0::X,0::X].values,transform=ccrs.PlateCarree(),units='inches',scale=10,width=0.04,alpha=0.85,zorder=30)
#ax.quiverkey(q,0.94,1.01,5,'5 kg m$^{-1}$ s$^{-1}$',fontproperties={'size':24})

slp_means_all=slp_days.mean(dim='time',skipna=True)
thickness_means_all=thickness_days.mean(dim='time',skipna=True)

#c1=ax.contour(thickness_mean.longitude,thickness_mean.latitude,thickness_mean/100.,linewidths=3.5,linestyles='--',transform=ccrs.PlateCarree(),colors=['b','b','b','b','b','r','r','r','r','r','r'],levels=np.arange(516,580,6))
c=ax.contour(slp_means_all.longitude,slp_means_all.latitude,slp_means_all/100.,linewidths=3.5,transform=ccrs.PlateCarree(),colors='k',levels=np.arange(960,1040,4))

props = dict(boxstyle='round', facecolor='wheat', alpha=1.0)
ax.text(0.009, 0.9865,'n='+str(len(dates_choose)), transform=ax.transAxes, fontsize=34,verticalalignment='top', bbox=props)
cbar.ax.tick_params(labelsize=32)
cbar.set_label('kg m$^{-1}$ s$^{-1}$',fontsize=34,rotation=90,labelpad=15)
plt.clabel(c,fmt='%.00f',fontsize=30)
#plt.clabel(c1,fmt='%.00f',fontsize=30)

fig.canvas.draw()

# Define gridline locations and draw the lines using cartopy's built-in gridliner:
xticks = [-135,-130,-125,-120,-115,-110,-105,-95,-85,-75,-65,-55,-45,-40,-35,-30,-25,-20]
yticks = [20, 25, 30, 35, 40, 45]
ax.gridlines(xlocs=xticks, ylocs=yticks,alpha=0.5)
ax.tick_params(labelsize=34)
# Label the end-points of the gridlines using the custom tick makers:
ax.xaxis.set_major_formatter(LONGITUDE_FORMATTER) 
ax.yaxis.set_major_formatter(LATITUDE_FORMATTER)
lambert_xticks(ax, xticks)
lambert_yticks(ax, yticks)
#ax.text(lonmax+15,clat,'mm/day',ha='right',va='center',rotation=270,fontsize=14)
plt.show()

#done: fall, spring, summer, winter

In [ ]:
fig.savefig(dir+'neusa_climo_event_trends_'+season+'5.png')

In [ ]:
proj_map = ccrs.PlateCarree()
clon=-70
clat=35
proj_map = ccrs.LambertConformal(central_longitude=clon, central_latitude=clat)
fig = plt.figure(figsize=(22,18))
ax=plt.subplot(1,1,1,projection=proj_map)
# *must* call draw in order to get the axis boundary used to add ticks:


#cax=ax.contourf(slopes_array_slp.longitude,slopes_array_slp.latitude,slopes_array_slp/100.,transform=ccrs.PlateCarree(),levels=np.arange(-0.3,0.33,0.03),extend='both',cmap=plt.cm.bwr)
#cbar=plt.colorbar(cax,pad=0,fraction=0.042)
#cbar.set_label('hPa yr$^{-1}$',fontsize=22,rotation=90,labelpad=0)

ax.coastlines(resolution='10m')
ax.add_feature(cfeature.STATES.with_scale('10m'),alpha=0.3)
ax.add_feature(cfeature.LAKES.with_scale('50m'))
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
    
ax.set_extent([-100,-45,19,55],crs=ccrs.PlateCarree())
#ax.set_title('ERA5 SLP and IVT trends',loc='left',fontsize=20)
ax.set_title('AR frequency',fontsize=52,pad=10)

Y=5
if 1==0:
    for i in range(int(slopes_array_slp.longitude.size/Y)):
        for j in range(int(slopes_array_slp.latitude.size/Y)):
            if sigs_array_slp[Y*j,Y*i]<=0.05:
                ax.plot(slopes_array_slp.longitude.values[Y*i],slopes_array_slp.latitude.values[Y*j],transform=ccrs.PlateCarree(),marker='o',color='w',mew=1.75,markersize=6,markerfacecolor="None")
            elif sigs_array_slp[Y*j,Y*i]<=0.10:
                ax.plot(slopes_array_slp.longitude.values[Y*i],slopes_array_slp.latitude.values[Y*j],transform=ccrs.PlateCarree(),marker='o',color='w',mew=0.5,markersize=6,markerfacecolor="None")

#for i in range(int(slopes_array.longitude.size/Y)):
#    for j in range(int(slopes_array.latitude.size/Y)):
#        if sigs_array_v[Y*j,Y*i]==1 or sigs_array_u[Y*j,Y*i]==1:
#            print('YES')
#            
#            ax.quiver(u1.longitude[Y*i],u1.longitude[Y*j],slopes_array_u[Y*j,Y*i],slopes_array_v[Y*j,Y*i])
#        else:
#            if sigs_array_v_10[Y*j,Y*i]==1 or sigs_array_u_10[Y*j,Y*i]==1:
#                print('YES')
#                ax.quiver(u1.longitude[Y*i],u1.longitude[Y*j],slopes_array_u[Y*j,Y*i],slopes_array_v[Y*j,Y*i])

sigs_array_ivtx_convert5=sigs_array_ivtx.where(sigs_array_ivtx<=0.05)
sigs_array_ivtx_convert5=sigs_array_ivtx_convert5/sigs_array_ivtx_convert5
sigs_array_ivty_convert5=sigs_array_ivty.where(sigs_array_ivty<=0.05)
sigs_array_ivty_convert5=sigs_array_ivty_convert5/sigs_array_ivty_convert5

sigs_array_ivtx_convert10=sigs_array_ivtx.where(sigs_array_ivtx<=0.10)
sigs_array_ivtx_convert10=sigs_array_ivtx_convert10/sigs_array_ivtx_convert10
sigs_array_ivty_convert10=sigs_array_ivty.where(sigs_array_ivty<=0.10)
sigs_array_ivty_convert10=sigs_array_ivty_convert10/sigs_array_ivty_convert10


sigs_array_u=sigs_array_ivtx_convert5
sigs_array_v=sigs_array_ivty_convert5
sigs_array_u10=sigs_array_ivtx_convert10
sigs_array_v10=sigs_array_ivty_convert10

slopes_array_u=slopes_array_ivtx
slopes_array_v=slopes_array_ivty

sig_uv=sigs_array_u+sigs_array_v
sig_uv10=sigs_array_u10+sigs_array_v10

sig_uv=sig_uv.fillna(0)
sig_uv10=sig_uv10.fillna(0)

X=8
slopes_array_u10_select=slopes_array_u.where(sig_uv10>0)
slopes_array_v10_select=slopes_array_v.where(sig_uv10>0)
q=ax.quiver(ivtx.longitude[0::X].values,ivtx.latitude[0::X].values,slopes_array_u10_select[0::X,0::X].values,slopes_array_v10_select[0::X,0::X].values,transform=ccrs.PlateCarree(),units='inches',color='k',scale=10,width=0.04,alpha=0.45,zorder=30)

slopes_array_u_select=slopes_array_u.where(sig_uv>0)
slopes_array_v_select=slopes_array_v.where(sig_uv>0)

q=ax.quiver(ivtx.longitude[0::X].values,ivtx.latitude[0::X].values,slopes_array_u_select[0::X,0::X].values,slopes_array_v_select[0::X,0::X].values,transform=ccrs.PlateCarree(),units='inches',color='k',scale=10,width=0.06,alpha=0.45,zorder=30)
ax.quiverkey(q,0.94,1.01,5,'5 kg m$^{-1}$ s$^{-1}$',fontproperties={'size':24})

slp_means_all=slp_days.mean(dim='time',skipna=True)
thickness_means_all=thickness_days.mean(dim='time',skipna=True)

#c1=ax.contour(thickness_mean.longitude,thickness_mean.latitude,thickness_mean/100.,linewidths=3.5,linestyles='--',transform=ccrs.PlateCarree(),colors=['b','b','b','b','b','r','r','r','r','r','r'],levels=np.arange(516,580,6))
c=ax.contour(slopes_array_slp.longitude,slopes_array_slp.latitude,slopes_array_slp/100.,linewidths=3.5,transform=ccrs.PlateCarree(),colors='k',levels=np.arange(-0.3,0.36,0.06),zorder=31)

props = dict(boxstyle='round', facecolor='wheat', alpha=1.0)
ax.text(0.009, 0.9865,'n='+str(len(dates_choose)), transform=ax.transAxes, fontsize=34,verticalalignment='top', bbox=props,zorder=34)
cbar.ax.tick_params(labelsize=32)
cbar.set_label('kg m$^{-1}$ s$^{-1}$',fontsize=34,rotation=90,labelpad=15)
ax.clabel(c,inline=True,fmt='%.02f',fontsize=30)
#plt.clabel(c1,fmt='%.00f',fontsize=30)

slopes_ar_pos=slopes_array_ar.where(slopes_array_ar>=0.001)
slopes_ar_neg=slopes_array_ar.where(slopes_array_ar<=-0.001)
cax=ax.contourf(slopes_ar_pos.longitude,slopes_ar_pos.latitude,slopes_ar_pos*100.,transform=ccrs.PlateCarree(),levels=np.arange(-1.4,1.6,0.2),extend='both',cmap=plt.cm.bwr_r)
cbar=plt.colorbar(cax,pad=0,fraction=0.03,aspect=60,orientation='horizontal')
cbar.set_label('AR probability (% yr$^{-1}$)',fontsize=34,labelpad=10)
cbar.ax.tick_params(labelsize=32)

cax=ax.contourf(slopes_ar_neg.longitude,slopes_ar_neg.latitude,slopes_ar_neg*100.,transform=ccrs.PlateCarree(),levels=np.arange(-1.4,1.6,0.2),extend='both',cmap=plt.cm.bwr_r)

fig.canvas.draw()

# Define gridline locations and draw the lines using cartopy's built-in gridliner:
xticks = [-135,-130,-125,-120,-115,-110,-105,-95,-85,-75,-65,-55,-45,-40,-35,-30,-25,-20]
yticks = [20, 25, 30, 35, 40, 45]
ax.gridlines(xlocs=xticks, ylocs=yticks,alpha=0.5)
ax.tick_params(labelsize=34)
# Label the end-points of the gridlines using the custom tick makers:
ax.xaxis.set_major_formatter(LONGITUDE_FORMATTER) 
ax.yaxis.set_major_formatter(LATITUDE_FORMATTER)
#lambert_xticks(ax, xticks)
lambert_yticks(ax, yticks)
#ax.text(lonmax+15,clat,'mm/day',ha='right',va='center',rotation=270,fontsize=14)
plt.show()

#done: fall, spring, summer, winter

In [ ]:
fig.savefig(dir+'neusa_climo_event_trends_'+season+'6a.png')